In [17]:
import torch
from nubo.acquisition import ExpectedImprovement, UpperConfidenceBound
from nubo.models import GaussianProcess, fit_gp
from nubo.optimisation import lbfgsb
from nubo.utils import gen_inputs
from gpytorch.likelihoods import GaussianLikelihood
from toy_function import Multimodal_1D
import matplotlib.pyplot as plt
from gpytorch.constraints import Interval

# test function
func = Multimodal_1D(minimise=True)
dims = func.dims
bounds = func.bounds

# training data
torch.manual_seed(1)
x_train = gen_inputs(num_points=2,
                     num_dims=1,
                     bounds=bounds,
                     num_lhd=2)
y_train = func(x_train)

# Bayesian optimisation loop
iters = 20

for iter in range(iters):
    
    # specify Gaussian process
    likelihood = GaussianLikelihood(noise_constraint=Interval(0.0, 0.0000001))
    gp = GaussianProcess(x_train, y_train, likelihood=likelihood)
    
    # fit Gaussian process
    fit_gp(x_train, y_train, gp=gp, likelihood=likelihood, lr=0.1, steps=200)

    # specify acquisition function
    acq = ExpectedImprovement(gp=gp, y_best=torch.max(y_train))
    # acq = UpperConfidenceBound(gp=gp, beta=16)

    # optimise acquisition function
    x_new, _ = lbfgsb(func=acq, bounds=bounds, num_starts=5, num_samples=1000)

    # evaluate new point
    y_new = func(x_new)
    
    # add to data
    x_train = torch.vstack((x_train, x_new))
    y_train = torch.hstack((y_train, y_new))

    # print new best
    if y_new > torch.max(y_train[:-1]):
        print(f"New best at evaluation {len(y_train)}: \t Inputs: {x_new.numpy().reshape(dims).round(4)}, \t Outputs: {-y_new.numpy().round(4)}")
    
    # compute GP for plotting
    x_plot = torch.linspace(start=-0.5, end=10.5, steps=1001, dtype=torch.double)

    gp.eval()

    pred = gp(x_plot)

    gp_means = pred.mean
    gp_vars = pred.variance.clamp_min(1e-10)
    gp_ci_upper = gp_means + torch.sqrt(gp_vars)*1.96
    gp_ci_lower = gp_means - torch.sqrt(gp_vars)*1.96
    true = func(x_plot)
    ucbs = -acq(torch.reshape(x_plot, (-1, 1))).detach().numpy()

    x_plot = x_plot.detach().numpy()
    gp_means = gp_means.detach().numpy()
    gp_ci_upper = gp_ci_upper.detach().numpy()
    gp_ci_lower = gp_ci_lower.detach().numpy()
    
    fig, ax1 = plt.subplots()
    ax1.plot(x_train[:-1], y_train[:-1], marker="o", linewidth=0, color="navy", label="Observations", zorder=6)
    ax1.axvline(float(x_train[-1]), color="red", linestyle="dotted", linewidth=1, label="Candidate", zorder=5)
    ax1.plot(x_plot, gp_means, color="firebrick", label="Prediction", zorder=4)
    ax1.plot(x_plot, true, color="black", linestyle="dashed", label="Truth", zorder=2)
    ax1.fill_between(x_plot, gp_ci_upper, gp_ci_lower, color="skyblue", label="Uncertainty", zorder=1)
    pos = ax1.get_position()
    ax1.set_position([pos.x0, pos.y0+pos.height*0.175, pos.width, pos.height * 0.85]) 

    ax2 = ax1.twinx() 
    ax2.fill_between(x_plot, ucbs, color="darkorange", label="EI", zorder=3, alpha=0.7)
    ax2.set_position([pos.x0, pos.y0+pos.height*0.175, pos.width, pos.height * 0.85])


    from matplotlib.patches import Patch
    from matplotlib.lines import Line2D

    legend_elements = [Line2D([0], [0], marker="o", linewidth=0, color="navy", label="Observations"),
                       Patch(color="darkorange", label="Acquisition", zorder=3, alpha=0.7),
                       Line2D([0], [0], color="firebrick", label="Prediction"),
                       Line2D([0], [0], color="red", linestyle="dotted", linewidth=1, label="Candidate"),
                       Patch(color="skyblue", label="Uncertainty"),
                       Line2D([0], [0], color="black", linestyle="dashed", label="Truth")]
                                              
    ax1.legend(handles=legend_elements, loc='lower center', ncol=3, bbox_to_anchor=(0.5, -0.35))
    plt.title(r"Expected improvement")
    ax1.set_ylabel("Output")
    ax2.set_ylabel("Expected improvement")
    ax1.set_xlabel("Input")
    plt.xlim((-0.5, 10.5))
    ax1.set_ylim((-15., 9.))
    ax2.set_ylim((0., 0.6))
    ax1.text(-0.35, 7.7, f"Iteration {iter+1}")
    plt.savefig(f"ei_{iter}")
    plt.clf()

# results
best_iter = int(torch.argmax(y_train))
print(f"Evaluation: {best_iter+1} \t Solution: {float(y_train[best_iter]):.4f}")


New best at evaluation 4: 	 Inputs: [2.3319], 	 Outputs: -1.6885000467300415
New best at evaluation 6: 	 Inputs: [2.015], 	 Outputs: -1.8193999528884888
New best at evaluation 10: 	 Inputs: [7.6207], 	 Outputs: -7.4141998291015625
New best at evaluation 11: 	 Inputs: [7.8413], 	 Outputs: -7.840700149536133
New best at evaluation 12: 	 Inputs: [7.9806], 	 Outputs: -7.9166998863220215
Evaluation: 12 	 Solution: 7.9167


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

...

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>